In [163]:
import html
import json
import requests

In [164]:
#URL = 'http://localhost:5001/api/v1/generate'
URL = 'http://localhost:5001/api/v1/chat'

In [165]:
available_functions = """
{
    "name": "go_to_location",
    "description": "Go to a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {
                "type": "number",
                "description": "The latitude to go to.",
            },
            "longitude": {
                "type": "number",
                "description": "The longitude to go to.",
            },
        },
        "required": ["latitude", "longitude"],
    },
},
{
    "name": "pan_in_direction",
    "description": "Pan in a given direction and distance in kilometers.",
    "parameters": {
        "type": "object",
        "properties": {
            "direction": {
                "type": "string",
                "description": "The direction to pan in. One of 'north', 'south', 'east', 'west', 'northwest', 'northeast', 'southwest', 'southeast'.",
            },
            "distance_in_kilometers": {
                "type": "number",
                "description": "The distance to pan in kilometers. If not provided, defaults to 1.",
            },
        },
        "required": ["direction"],
    },
},
{
    "name": "zoom_in",
    "description": "Zoom in by a given number of zoom levels.",
    "parameters": {
        "type": "object",
        "properties": {
            "zoom_levels": {
                "type": "number",
                "description": "The number of zoom levels to zoom in. If not provided, defaults to 1.",
            },
        },
    },
    "required": ["zoom_levels"],
},
{
    "name": "zoom_out",
    "description": "Zoom out by a given number of zoom levels.",
    "parameters": {
        "type": "object",
        "properties": {
            "zoom_levels": {
                "type": "number",
                "description": "The number of zoom levels to zoom out. If not provided, defaults to 1.",
            },
        },
    },
    "required": ["zoom_levels"],
}
"""

In [166]:
system_message = f"""As an AI assistant, please select the most suitable function and parameters 
from the list of available functions below, based on the user's input. Provide your response in JSON format."""

In [167]:
def send_message(message, system_message, function_descriptions, temperature=0.0):
    payload = f"""{system_message}\nInput: {message}\nAvailable functions:\n{function_descriptions}"""
    history = {'internal': [], 'visible': []}
    request = {
        'user_input': payload,
        'history': history,
        'temperature': temperature,
    }
    response = requests.post(URL, json=request)
    if response.status_code == 200:
        result = response.json()['results'][0]['history']
        response_data = html.unescape(result['visible'][-1][1])
        response_message = json.loads(response_data.strip('\n'))
        response = {"response": response_message}
        return response

In [168]:
message = "go to paris"

In [169]:
send_message(message, system_message, available_functions)

{'response': {'function': 'go_to_location',
  'parameters': {'latitude': 48.8567, 'longitude': 2.3522}}}

In [170]:
message = "zoom in"

In [171]:
send_message(message, system_message, available_functions)

{'response': {'function': 'zoom_in', 'parameters': {'zoom_levels': 2}}}

In [172]:
message = "pan north by 30km"

In [173]:
send_message(message, system_message, available_functions)

{'response': {'function': 'pan_in_direction',
  'parameters': {'direction': 'north', 'distance_in_kilometers': 30}}}